In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
%matplotlib inline  
import plotly.offline as py
import plotly.express as px

In [2]:
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs,init_notebook_mode,plot, iplot
import plotly.tools as tls
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)

In [3]:
from mlxtend.preprocessing import minmax_scaling 

In [4]:
import tensorflow as tf

In [5]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [7]:
data = pd.read_csv("diabetes.csv")

In [8]:
display(data.info(),data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


None

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [10]:
data[['Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age']] = data[['Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age']].replace(0,np.NaN)    

In [11]:
data.isnull().sum()

Pregnancies                   0
Glucose                       5
BloodPressure                35
SkinThickness               227
Insulin                     374
BMI                          11
DiabetesPedigreeFunction      0
Age                           0
Outcome                       0
dtype: int64

In [12]:
def missing_plot(dataset, key) :
    null_feat = pd.DataFrame(len(dataset[key]) - dataset.isnull().sum(), columns = ['Count'])
    percentage_null = pd.DataFrame((dataset.isnull().sum())/len(dataset[key])*100, columns = ['Count'])
    percentage_null = percentage_null.round(2)

    trace = go.Bar(x = null_feat.index, y = null_feat['Count'] ,opacity = 0.8, text = percentage_null['Count'],  textposition = 'auto',marker=dict(color = '#7EC0EE',
            line=dict(color='#000000',width=1.5)))

    layout = dict(title =  "Missing Values (count & %)")

    fig = dict(data = [trace], layout=layout)
    py.iplot(fig)

In [13]:
missing_plot(data,'Outcome')

In [14]:
def find_median(var):
    temp = data[data[var].notnull()]
    temp = data[[var,'Outcome']].groupby('Outcome')[[var]].median().reset_index()
    return temp

In [19]:
def density_plot(var,size_bin):
    tmp1 = Diabeties[var]
    tmp2 = Healthy[var]
    
    hist_data = [tmp1,tmp2]
    labels = ['Diabeties','Healthy']
    color = ['skyblue','indigo']
    fig = ff.create_distplot(hist_data,labels,colors = color,show_hist=True,bin_size=size_bin,curve_type='kde')
    
    fig['layout'].update(title = var)

    py.iplot(fig, filename = 'Density plot')
    

In [21]:
find_median('Insulin')

,Outcome,Insulin
0,0,102.5
1,1,169.5


In [22]:
data.loc[(data['Outcome'] == 0) & (data['Insulin'].isnull()), 'Insulin'] = 102.5
data.loc[(data['Outcome'] == 1) & (data['Insulin'].isnull()), 'Insulin'] = 169.5

In [24]:
find_median('SkinThickness')

,Outcome,SkinThickness
0,0,27.0
1,1,32.0


In [25]:
data.loc[(data['Outcome'] == 0) & (data['SkinThickness'].isnull()), 'SkinThickness'] = 27.0
data.loc[(data['Outcome'] == 1) & (data['SkinThickness'].isnull()), 'SkinThickness'] = 32.0

In [26]:
find_median('BloodPressure')

,Outcome,BloodPressure
0,0,70.0
1,1,74.5


In [27]:
data.loc[(data['Outcome'] == 0) & (data['BloodPressure'].isnull()), 'BloodPressure'] = 27.0
data.loc[(data['Outcome'] == 1) & (data['BloodPressure'].isnull()), 'BloodPressure'] = 32.0

In [28]:
find_median('BMI')

,Outcome,BMI
0,0,30.1
1,1,34.3


In [29]:
data.loc[(data['Outcome'] == 0) & (data['BMI'].isnull()), 'BMI'] = 30.1
data.loc[(data['Outcome'] == 1) & (data['BMI'].isnull()), 'BMI'] = 34.3

In [30]:
find_median('Glucose')

,Outcome,Glucose
0,0,107.0
1,1,140.0


In [31]:
data.loc[(data['Outcome'] == 0) & (data['Glucose'].isnull()) , 'Glucose'] = 107.0
data.loc[(data['Outcome'] == 1) & (data['Glucose'].isnull()) , 'Glucose'] = 140.0

In [32]:
display(data.isnull().sum())

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [33]:
def correlation_plot():
    #correlation
    correlation = data.corr()
    #tick labels
    matrix_cols = correlation.columns.tolist()
    #convert to array
    corr_array  = np.array(correlation)
    trace = go.Heatmap(z = corr_array,
                       x = matrix_cols,
                       y = matrix_cols,
                       colorscale='Viridis',
                       colorbar   = dict() 
                      )
    layout = go.Layout(dict(title = 'Correlation Matrix for variables',
                            #autosize = False,
                            #height  = 1400,
                            #width   = 1600,
                            margin  = dict(r = 0 ,l = 100,
                                           t = 0,b = 100,
                                         ),
                            yaxis   = dict(tickfont = dict(size = 9)),
                            xaxis   = dict(tickfont = dict(size = 9)),
                           )
                      )
    fig = go.Figure(data = [trace],layout = layout)
    py.iplot(fig)

In [34]:
correlation_plot()

In [35]:
def plot_feat1_feat2(feat1, feat2) :  
    D = data[(data['Outcome'] != 0)]
    H = data[(data['Outcome'] == 0)]
    trace0 = go.Scatter(
        x = D[feat1],
        y = D[feat2],
        name = 'diabetic',
        mode = 'markers', 
        opacity=0.8,
        marker = dict(color = 'lightskyblue',
            line = dict(
                width = 1)))

    trace1 = go.Scatter(
        x = H[feat1],
        y = H[feat2],
        name = 'healthy',
        opacity=0.8,
        mode = 'markers',
        marker = dict(color = 'indigo',
            line = dict(
                width = 1)))

    layout = dict(title = feat1 +" "+"vs"+" "+ feat2,
                  yaxis = dict(title = feat2,zeroline = False),
                  xaxis = dict(title = feat1, zeroline = False)
                 )

    plots = [trace0, trace1]

    fig = dict(data = plots, layout=layout)
    py.iplot(fig)

In [36]:
plot_feat1_feat2('Pregnancies', 'Age')

In [37]:
plot_feat1_feat2('Glucose', 'Insulin')

In [38]:
scaled_data = minmax_scaling(data,columns=['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age'])

In [39]:
def build_model():
    model = tf.keras.Sequential([
    tf.keras.layers.Dense(8, activation='relu', input_shape=[len(scaled_data.keys())]),
    tf.keras.layers.Dense(4, activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
  ])

    optimizer = tf.keras.optimizers.RMSprop(0.01)

    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

model = build_model()

In [40]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 72        
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 36        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 5         
Total params: 113
Trainable params: 113
Non-trainable params: 0
_________________________________________________________________


In [41]:
EPOCHS = 1000

history = model.fit(scaled_data, data['Outcome'],epochs=EPOCHS, validation_split=0.2, verbose=2)

Epoch 1/1000
20/20 - 1s - loss: 0.6485 - accuracy: 0.6531 - val_loss: 0.6162 - val_accuracy: 0.6429
Epoch 2/1000
20/20 - 0s - loss: 0.6025 - accuracy: 0.6612 - val_loss: 0.5790 - val_accuracy: 0.6883
Epoch 3/1000
20/20 - 0s - loss: 0.5684 - accuracy: 0.7068 - val_loss: 0.5511 - val_accuracy: 0.7662
Epoch 4/1000
20/20 - 0s - loss: 0.5280 - accuracy: 0.7345 - val_loss: 0.5370 - val_accuracy: 0.8052
Epoch 5/1000
20/20 - 0s - loss: 0.5049 - accuracy: 0.7655 - val_loss: 0.4917 - val_accuracy: 0.7922
Epoch 6/1000
20/20 - 0s - loss: 0.4808 - accuracy: 0.7687 - val_loss: 0.4797 - val_accuracy: 0.7468
Epoch 7/1000
20/20 - 0s - loss: 0.4631 - accuracy: 0.7850 - val_loss: 0.4680 - val_accuracy: 0.7987
Epoch 8/1000
20/20 - 0s - loss: 0.4553 - accuracy: 0.7964 - val_loss: 0.4735 - val_accuracy: 0.7922
Epoch 9/1000
20/20 - 0s - loss: 0.4515 - accuracy: 0.7915 - val_loss: 0.4618 - val_accuracy: 0.7597
Epoch 10/1000
20/20 - 0s - loss: 0.4475 - accuracy: 0.7834 - val_loss: 0.5166 - val_accuracy: 0.7143

Epoch 83/1000
20/20 - 0s - loss: 0.3269 - accuracy: 0.8664 - val_loss: 0.3715 - val_accuracy: 0.8182
Epoch 84/1000
20/20 - 0s - loss: 0.3238 - accuracy: 0.8713 - val_loss: 0.3686 - val_accuracy: 0.8312
Epoch 85/1000
20/20 - 0s - loss: 0.3216 - accuracy: 0.8746 - val_loss: 0.3781 - val_accuracy: 0.8052
Epoch 86/1000
20/20 - 0s - loss: 0.3223 - accuracy: 0.8697 - val_loss: 0.4137 - val_accuracy: 0.8247
Epoch 87/1000
20/20 - 0s - loss: 0.3286 - accuracy: 0.8664 - val_loss: 0.3605 - val_accuracy: 0.8247
Epoch 88/1000
20/20 - 0s - loss: 0.3181 - accuracy: 0.8599 - val_loss: 0.3878 - val_accuracy: 0.8182
Epoch 89/1000
20/20 - 0s - loss: 0.3118 - accuracy: 0.8746 - val_loss: 0.3655 - val_accuracy: 0.8831
Epoch 90/1000
20/20 - 0s - loss: 0.3312 - accuracy: 0.8632 - val_loss: 0.3560 - val_accuracy: 0.8312
Epoch 91/1000
20/20 - 0s - loss: 0.3190 - accuracy: 0.8795 - val_loss: 0.5088 - val_accuracy: 0.7662
Epoch 92/1000
20/20 - 0s - loss: 0.3277 - accuracy: 0.8583 - val_loss: 0.3509 - val_accurac

Epoch 164/1000
20/20 - 0s - loss: 0.2894 - accuracy: 0.8876 - val_loss: 0.3470 - val_accuracy: 0.8442
Epoch 165/1000
20/20 - 0s - loss: 0.2961 - accuracy: 0.8713 - val_loss: 0.3451 - val_accuracy: 0.8377
Epoch 166/1000
20/20 - 0s - loss: 0.2973 - accuracy: 0.8664 - val_loss: 0.3546 - val_accuracy: 0.8377
Epoch 167/1000
20/20 - 0s - loss: 0.2849 - accuracy: 0.8860 - val_loss: 0.4061 - val_accuracy: 0.8377
Epoch 168/1000
20/20 - 0s - loss: 0.2940 - accuracy: 0.8811 - val_loss: 0.3504 - val_accuracy: 0.8377
Epoch 169/1000
20/20 - 0s - loss: 0.2822 - accuracy: 0.8941 - val_loss: 0.3698 - val_accuracy: 0.8117
Epoch 170/1000
20/20 - 0s - loss: 0.2964 - accuracy: 0.8811 - val_loss: 0.3676 - val_accuracy: 0.8377
Epoch 171/1000
20/20 - 0s - loss: 0.2918 - accuracy: 0.8876 - val_loss: 0.3832 - val_accuracy: 0.8182
Epoch 172/1000
20/20 - 0s - loss: 0.2914 - accuracy: 0.8795 - val_loss: 0.3758 - val_accuracy: 0.8247
Epoch 173/1000
20/20 - 0s - loss: 0.2883 - accuracy: 0.8844 - val_loss: 0.3684 - v

Epoch 245/1000
20/20 - 0s - loss: 0.2918 - accuracy: 0.8746 - val_loss: 0.3774 - val_accuracy: 0.8442
Epoch 246/1000
20/20 - 0s - loss: 0.2778 - accuracy: 0.8876 - val_loss: 0.3927 - val_accuracy: 0.8182
Epoch 247/1000
20/20 - 0s - loss: 0.2821 - accuracy: 0.8860 - val_loss: 0.3525 - val_accuracy: 0.8701
Epoch 248/1000
20/20 - 0s - loss: 0.2763 - accuracy: 0.8893 - val_loss: 0.3745 - val_accuracy: 0.8506
Epoch 249/1000
20/20 - 0s - loss: 0.2861 - accuracy: 0.8795 - val_loss: 0.3549 - val_accuracy: 0.8571
Epoch 250/1000
20/20 - 0s - loss: 0.2834 - accuracy: 0.8941 - val_loss: 0.3843 - val_accuracy: 0.8442
Epoch 251/1000
20/20 - 0s - loss: 0.2857 - accuracy: 0.8893 - val_loss: 0.3877 - val_accuracy: 0.8377
Epoch 252/1000
20/20 - 0s - loss: 0.2757 - accuracy: 0.8844 - val_loss: 0.3776 - val_accuracy: 0.8442
Epoch 253/1000
20/20 - 0s - loss: 0.2839 - accuracy: 0.8811 - val_loss: 0.3575 - val_accuracy: 0.8377
Epoch 254/1000
20/20 - 0s - loss: 0.2838 - accuracy: 0.8893 - val_loss: 0.3880 - v

Epoch 326/1000
20/20 - 0s - loss: 0.2816 - accuracy: 0.8860 - val_loss: 0.3964 - val_accuracy: 0.8312
Epoch 327/1000
20/20 - 0s - loss: 0.2810 - accuracy: 0.8795 - val_loss: 0.3688 - val_accuracy: 0.8442
Epoch 328/1000
20/20 - 0s - loss: 0.2790 - accuracy: 0.8811 - val_loss: 0.3627 - val_accuracy: 0.8571
Epoch 329/1000
20/20 - 0s - loss: 0.2759 - accuracy: 0.8876 - val_loss: 0.3873 - val_accuracy: 0.8117
Epoch 330/1000
20/20 - 0s - loss: 0.2685 - accuracy: 0.8909 - val_loss: 0.3794 - val_accuracy: 0.8442
Epoch 331/1000
20/20 - 0s - loss: 0.2796 - accuracy: 0.8827 - val_loss: 0.3602 - val_accuracy: 0.8506
Epoch 332/1000
20/20 - 0s - loss: 0.2740 - accuracy: 0.8909 - val_loss: 0.3668 - val_accuracy: 0.8506
Epoch 333/1000
20/20 - 0s - loss: 0.2741 - accuracy: 0.8844 - val_loss: 0.3909 - val_accuracy: 0.8182
Epoch 334/1000
20/20 - 0s - loss: 0.2761 - accuracy: 0.8893 - val_loss: 0.3664 - val_accuracy: 0.8442
Epoch 335/1000
20/20 - 0s - loss: 0.2753 - accuracy: 0.8974 - val_loss: 0.4054 - v

...

Epoch 650/1000
20/20 - 0s - loss: 0.2678 - accuracy: 0.8909 - val_loss: 0.3981 - val_accuracy: 0.8377
Epoch 651/1000
20/20 - 0s - loss: 0.2676 - accuracy: 0.8844 - val_loss: 0.3671 - val_accuracy: 0.8442
Epoch 652/1000
20/20 - 0s - loss: 0.2697 - accuracy: 0.9007 - val_loss: 0.4145 - val_accuracy: 0.8377
Epoch 653/1000
20/20 - 0s - loss: 0.2687 - accuracy: 0.8860 - val_loss: 0.3745 - val_accuracy: 0.8442
Epoch 654/1000
20/20 - 0s - loss: 0.2694 - accuracy: 0.8909 - val_loss: 0.3846 - val_accuracy: 0.8442
Epoch 655/1000
20/20 - 0s - loss: 0.2733 - accuracy: 0.8876 - val_loss: 0.3723 - val_accuracy: 0.8377
Epoch 656/1000
20/20 - 0s - loss: 0.2708 - accuracy: 0.8893 - val_loss: 0.3636 - val_accuracy: 0.8506
Epoch 657/1000
20/20 - 0s - loss: 0.2606 - accuracy: 0.9023 - val_loss: 0.3983 - val_accuracy: 0.8636
Epoch 658/1000
20/20 - 0s - loss: 0.2671 - accuracy: 0.8958 - val_loss: 0.3936 - val_accuracy: 0.8182
Epoch 659/1000
20/20 - 0s - loss: 0.2648 - accuracy: 0.8941 - val_loss: 0.3972 - v

Epoch 731/1000
20/20 - 0s - loss: 0.2674 - accuracy: 0.8958 - val_loss: 0.3842 - val_accuracy: 0.8377
Epoch 732/1000
20/20 - 0s - loss: 0.2633 - accuracy: 0.8941 - val_loss: 0.4448 - val_accuracy: 0.7922
Epoch 733/1000
20/20 - 0s - loss: 0.2641 - accuracy: 0.8941 - val_loss: 0.4103 - val_accuracy: 0.8442
Epoch 734/1000
20/20 - 0s - loss: 0.2668 - accuracy: 0.8941 - val_loss: 0.3661 - val_accuracy: 0.8442
Epoch 735/1000
20/20 - 0s - loss: 0.2734 - accuracy: 0.8909 - val_loss: 0.3917 - val_accuracy: 0.8312
Epoch 736/1000
20/20 - 0s - loss: 0.2603 - accuracy: 0.8990 - val_loss: 0.4235 - val_accuracy: 0.8052
Epoch 737/1000
20/20 - 0s - loss: 0.2677 - accuracy: 0.8990 - val_loss: 0.4125 - val_accuracy: 0.8182
Epoch 738/1000
20/20 - 0s - loss: 0.2654 - accuracy: 0.8876 - val_loss: 0.4305 - val_accuracy: 0.8312
Epoch 739/1000
20/20 - 0s - loss: 0.2645 - accuracy: 0.8958 - val_loss: 0.3786 - val_accuracy: 0.8442
Epoch 740/1000
20/20 - 0s - loss: 0.2588 - accuracy: 0.8974 - val_loss: 0.4338 - v

Epoch 812/1000
20/20 - 0s - loss: 0.2658 - accuracy: 0.8941 - val_loss: 0.4108 - val_accuracy: 0.8182
Epoch 813/1000
20/20 - 0s - loss: 0.2640 - accuracy: 0.8941 - val_loss: 0.4703 - val_accuracy: 0.8247
Epoch 814/1000
20/20 - 0s - loss: 0.2647 - accuracy: 0.8909 - val_loss: 0.4352 - val_accuracy: 0.8052
Epoch 815/1000
20/20 - 0s - loss: 0.2675 - accuracy: 0.8958 - val_loss: 0.4445 - val_accuracy: 0.7922
Epoch 816/1000
20/20 - 0s - loss: 0.2649 - accuracy: 0.8958 - val_loss: 0.4001 - val_accuracy: 0.8377
Epoch 817/1000
20/20 - 0s - loss: 0.2717 - accuracy: 0.8925 - val_loss: 0.4940 - val_accuracy: 0.8247
Epoch 818/1000
20/20 - 0s - loss: 0.2615 - accuracy: 0.9007 - val_loss: 0.4060 - val_accuracy: 0.8312
Epoch 819/1000
20/20 - 0s - loss: 0.2599 - accuracy: 0.8925 - val_loss: 0.4321 - val_accuracy: 0.8312
Epoch 820/1000
20/20 - 0s - loss: 0.2700 - accuracy: 0.8876 - val_loss: 0.3835 - val_accuracy: 0.8442
Epoch 821/1000
20/20 - 0s - loss: 0.2622 - accuracy: 0.9007 - val_loss: 0.3869 - v

Epoch 893/1000
20/20 - 0s - loss: 0.2678 - accuracy: 0.8941 - val_loss: 0.3902 - val_accuracy: 0.8377
Epoch 894/1000
20/20 - 0s - loss: 0.2614 - accuracy: 0.8958 - val_loss: 0.4189 - val_accuracy: 0.8182
Epoch 895/1000
20/20 - 0s - loss: 0.2819 - accuracy: 0.8941 - val_loss: 0.3907 - val_accuracy: 0.8571
Epoch 896/1000
20/20 - 0s - loss: 0.2646 - accuracy: 0.8958 - val_loss: 0.3819 - val_accuracy: 0.8442
Epoch 897/1000
20/20 - 0s - loss: 0.2640 - accuracy: 0.9007 - val_loss: 0.3803 - val_accuracy: 0.8442
Epoch 898/1000
20/20 - 0s - loss: 0.2614 - accuracy: 0.8941 - val_loss: 0.3900 - val_accuracy: 0.8377
Epoch 899/1000
20/20 - 0s - loss: 0.2625 - accuracy: 0.8941 - val_loss: 0.3948 - val_accuracy: 0.8312
Epoch 900/1000
20/20 - 0s - loss: 0.2674 - accuracy: 0.8941 - val_loss: 0.3950 - val_accuracy: 0.8312
Epoch 901/1000
20/20 - 0s - loss: 0.2707 - accuracy: 0.8844 - val_loss: 0.4113 - val_accuracy: 0.8247
Epoch 902/1000
20/20 - 0s - loss: 0.2636 - accuracy: 0.8876 - val_loss: 0.4255 - v

Epoch 974/1000
20/20 - 0s - loss: 0.2635 - accuracy: 0.8974 - val_loss: 0.3815 - val_accuracy: 0.8377
Epoch 975/1000
20/20 - 0s - loss: 0.2672 - accuracy: 0.8893 - val_loss: 0.4445 - val_accuracy: 0.8182
Epoch 976/1000
20/20 - 0s - loss: 0.2632 - accuracy: 0.8941 - val_loss: 0.4370 - val_accuracy: 0.8182
Epoch 977/1000
20/20 - 0s - loss: 0.2646 - accuracy: 0.9007 - val_loss: 0.4211 - val_accuracy: 0.8442
Epoch 978/1000
20/20 - 0s - loss: 0.2604 - accuracy: 0.8974 - val_loss: 0.4373 - val_accuracy: 0.8117
Epoch 979/1000
20/20 - 0s - loss: 0.2679 - accuracy: 0.8958 - val_loss: 0.4053 - val_accuracy: 0.8377
Epoch 980/1000
20/20 - 0s - loss: 0.2667 - accuracy: 0.8958 - val_loss: 0.4491 - val_accuracy: 0.8247
Epoch 981/1000
20/20 - 0s - loss: 0.2639 - accuracy: 0.8925 - val_loss: 0.3945 - val_accuracy: 0.8247
Epoch 982/1000
20/20 - 0s - loss: 0.2641 - accuracy: 0.8909 - val_loss: 0.4240 - val_accuracy: 0.8182
Epoch 983/1000
20/20 - 0s - loss: 0.2636 - accuracy: 0.8958 - val_loss: 0.3831 - v

In [42]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

,loss,accuracy,val_loss,val_accuracy,epoch
995,0.263962,0.892508,0.404225,0.831169,995
996,0.268534,0.890879,0.408825,0.824675,996
997,0.263245,0.894137,0.413756,0.831169,997
998,0.268077,0.895765,0.429661,0.844156,998
999,0.264094,0.894137,0.404038,0.831169,999


In [43]:
acc = (hist['accuracy'].tail().sum())*100/5 
val_acc = (hist['val_accuracy'].tail().sum())*100/5 

print("Training Accuracy = {}% and Validation Accuracy= {}%".format(acc,val_acc))

Training Accuracy = 89.34853315353394% and Validation Accuracy= 83.2467532157898%
